## install mecab on mac
```
brew install mecab mecab-ipadic  
pip install mecab-python3
```

## install neologd
```
git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
cd mecab-ipadic-neologd
./bin/install-mecab-ipadic-neologd -n
```

## get Japanese articles
livedoor ニュースコーパス

```
wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
tar xvzf ldcc-20140209.tar.gz
```

In [1]:
from urllib import request 
from pathlib import Path
import numpy as np
import tqdm
import re
import MeCab
from gensim import corpora, models

In [2]:
mecab = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/")

In [3]:
[line.split("\t") for line in mecab.parse("今日もがんばるぞい！").split("\n")]

[['今日', 'キョウ', '今日', '名詞-副詞可能', '', ''],
 ['も', 'モ', 'も', '助詞-係助詞', '', ''],
 ['がん', 'ガン', 'がん', '名詞-一般', '', ''],
 ['ばる', 'バル', 'バル', '名詞-一般', '', ''],
 ['ぞい', 'ゾイ', 'ぞい', '名詞-接尾-一般', '', ''],
 ['！', '！', '！', '記号-一般', '', ''],
 ['EOS'],
 ['']]

In [4]:
[line.split("\t") for line in mecab.parse("今日も一日がんばるぞい！").split("\n")]

[['今日も一日がんばるぞい', 'キョウモイチニチガンバルゾイ', '今日も一日がんばるぞい', '名詞-固有名詞-一般', '', ''],
 ['！', '！', '！', '記号-一般', '', ''],
 ['EOS'],
 ['']]

In [5]:
res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt")
stopwords = [line.decode("utf-8").strip() for line in res]

In [56]:
def tokenizer(text):
    l = [line.split("\t") for line in mecab.parse(text).split("\n")]
    res = [i[0] for i in l 
                   if len(i) >=4 
                       and ("名詞" in i[3] or "動詞" in i[3] or "形容詞" in i[3] )
                       and "数" not in i[3] and "助動詞" not in i[3] 
                       #and not re.search("[0-9]", i[0])
                       and i[0] not in stopwords
            ]
    return res

In [57]:
tokenizer("認めたくないものだな。自分自身の若さ故の過ちというものを。")

['認め', '自分自身', '若さ故の過ち']

In [58]:
doc_path = "./text/"
doc_dir = Path(doc_path)
dirs = [i for i in doc_dir.iterdir() if i.is_dir()]
dirs

[PosixPath('text/dokujo-tsushin'),
 PosixPath('text/it-life-hack'),
 PosixPath('text/kaden-channel'),
 PosixPath('text/livedoor-homme'),
 PosixPath('text/movie-enter'),
 PosixPath('text/peachy'),
 PosixPath('text/smax'),
 PosixPath('text/sports-watch'),
 PosixPath('text/topic-news')]

In [59]:
articles = [a for categ in dirs for a in categ.iterdir()]

In [60]:
len(articles)

7376

In [61]:
def read_doc(doc_id):
    with articles[doc_id].open() as f:
        print(f.read())

In [178]:
read_doc(5000)

http://news.livedoor.com/article/detail/6590044/
2012-05-24T12:55:00+0900
ソニー、Sony TabletシリーズをAndroid 4.0にOSバージョンアップするソフトウェアアップデートを提供開始
本日24日13時から提供開始！ 

ソニーは18日、同社のAndroid搭載タブレット「Sony Tablet」シリーズにおいて最新プラットフォーム「Android 4.0（開発コード名：IceCream Sandwich；ICS）」へOSバージョンアップするためのシステムソフトウェアアップデートを2012年5月24日（木）13：00から提供開始すると発表しています。

今回のアップデートは、Sony Tablet SシリーズおよびSony Tablet Pシリーズを対象に、OSをAndroid 4.0.3へバージョンアップします。当初は4月下旬に予定されていましたが、ソフトウェア開発に時間を要するとして4月27日に延期することが発表されていました。

OSバージョンアップ以外にも“Sony Tablet”独自の機能が追加されるとのことです。



主なアップデート内容は以下の通り。

＜Android 4.0.3に伴う機能追加＞

1. 最新版のAndroid 4.0.3にアップデートされることにより、Android 4.0対応スマートフォンとユーザーインターフェースが共通化します。
2. ロック画面からのカメラの直接起動やパノラマ写真の撮影、「ギャラリー」アプリケーションで画像の編集が可能になるなど、写真・動画についての機能が向上します。


＜“Sony Tablet”独自の機能追加＞
1. ソニー製BDレコーダー※1と連携してテレビ番組の視聴が可能
ソニー製のBDレコーダー※1で録画した番組や放送中の番組を“Sony Tablet”上で視聴することが可能になります。家庭内のネットワークに接続することで、テレビのない寝室やキッチン、書斎など、場所を選ばず家中でテレビ番組をハイビジョン（720p）で楽しむことができます。
新しくソニーが提供するアプリケーション「RECOPLA（レコプラ）」※3を、“Sony Tablet”上の「Select App」※4、またはGoogle Playストアから無

In [63]:
docs = []
for a in tqdm.tqdm(articles):
    with a.open() as f:
        f.readline()
        f.readline()
        docs.append(tokenizer(f.read()))

100%|██████████| 7376/7376 [00:31<00:00, 233.01it/s]


In [180]:
docs[5000][:8]

['ソニー', 'Sony Tablet', 'シリーズ', 'Android 4.0', 'OS', 'バージョンアップ', 'する', 'ソフトウェア']

In [65]:
d = corpora.Dictionary(docs)

stopwordsで排除しきれなかった邪魔な単語を処理

In [66]:
d.filter_extremes(no_below=5, no_above=0.2)
# nob_below以下の個数の単語を無視
# no_aboveの割合以上に出てくる単語を無視

d.compactify()
# idを振り直してコンパクトにする。

In [67]:
dic_num = len(d)
dic_num

26209

In [181]:
d.doc2bow(docs[5000][:20])

[(1309, 1),
 (1347, 1),
 (3314, 1),
 (5654, 1),
 (6351, 1),
 (7443, 1),
 (12220, 1),
 (13546, 1),
 (13912, 1),
 (14557, 1),
 (15594, 2),
 (16221, 1),
 (16721, 2),
 (16858, 1),
 (16920, 1),
 (17334, 1),
 (19119, 1)]

(id, num in a doc)

In [182]:
d[1309] # get word from id

'搭載'

In [71]:
corpus = [d.doc2bow(w) for w in docs]

In [72]:
corpus_words = sum(count for doc in corpus for id, count in doc)
corpus_words

1496217

In [73]:
num_topics = 100

In [151]:
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=d, num_topics=num_topics, iterations=1000)

In [183]:
sorted(lda.get_document_topics(corpus[5000]), key=lambda t:t[1], reverse=True)

[(58, 0.28008868786636115),
 (63, 0.18262631077437469),
 (69, 0.15614527821574345),
 (36, 0.10985682904192787),
 (8, 0.095077871815779882),
 (6, 0.068057259739714188),
 (57, 0.048950289885001788),
 (66, 0.015136483497717465),
 (51, 0.011141093068924806),
 (4, 0.010641537644589848)]

In [184]:
def get_topic_words(topic_id):
    for t in lda.get_topic_terms(topic_id):
        print("{}: {}".format(d[t[0]], t[1]))

In [186]:
get_topic_words(58)

D: 0.025643587634950966
ソフトウェア: 0.02262133739417123
スマートフォン: 0.017817280001601692
対応: 0.014450307899772643
更新: 0.01384202012381848
充電: 0.012517615988070755
NTTドコモ: 0.01096637888578934
SC: 0.010685212388917037
LTE: 0.009881178198168656
Xi: 0.00897604785480875


## see on tensorboard

http://projector.tensorflow.org/  
ref: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Tensorboard_visualizations.ipynb


In [ ]:
all_topics = lda.get_document_topics(corpus, minimum_probability=0)

In [ ]:
with open('doc_lda_tensor.tsv','w') as w:
    for doc_topics in all_topics:
        for topics in doc_topics:
            w.write(str(topics[1])+ "\t")
        w.write("\n")    

In [ ]:
meta = [str(a).split("/") for a in articles]

In [ ]:
meta[0]

In [ ]:
with open('doc_lda_metadata.tsv','w') as w:
    w.write('Titles\tGenres\n')
    for m in meta:
        w.write("%s\t%s\n" % (m[1][:2], m[1]))

## perplexity

In [ ]:
lda.log_perplexity(corpus)

Estimate the variational bound of documents from corpus: E_q[log p(corpus)] - E_q[log q(corpus)]

In [ ]:
bound = lda.bound(corpus)/corpus_words
bound #= lda.log_perplexity(corpus)

In [ ]:
perplexity = np.exp2(-bound ) # 2^(-bound per words)

In [ ]:
lda.top_topics(corpus=corpus, num_words=10)

## 類似度

In [156]:
doc_topics = np.array([lda[c] for c in corpus])

In [157]:
doc_topics[1]

[(5, 0.13031681195806283),
 (10, 0.027911956077360991),
 (13, 0.60346018382798394),
 (33, 0.079328600077435196),
 (54, 0.023118769644709325),
 (59, 0.096518251134575572),
 (97, 0.022946814112145997)]

In [158]:
doc_topics.shape

(7376,)

In [159]:
dense = np.zeros( (len(doc_topics), num_topics), float)

for doc_id, t in enumerate(doc_topics):
    for topic_id, weight in t:
        dense[doc_id, topic_id] = weight

例えば
$$
\text{doc_id(7376)}
\underbrace{
\begin{pmatrix}
0 & 0 & weight & \cdots & 0\\
0 & 0 & 0 & \cdots & 0\\
0 & weight & 0 & \cdots & 0\\
\vdots & \vdots & \vdots & \vdots & \vdots\\
0 & 0 & 0 & \cdots & 0
\end{pmatrix}
}_{\text{topics(100)}}
$$

In [160]:
from scipy.spatial import distance
pairwise = distance.squareform(distance.pdist(dense, metric="cosine"))

$$
1 - \frac{u \cdot v}
         {{||u||}_2 {||v||}_2}
$$

In [161]:
pairwise.shape

(7376, 7376)

In [162]:
def similar_to(doc_id, num=5, distance=False):
    doc = pairwise[doc_id]
    ids = np.argsort(doc)[1: num+1] # the most similar is self.
    if distance:
        dist = np.sort(doc)[1: num+1]
        return [(x,y) for x, y in zip(ids, dist)]
    else:
        return ids

In [177]:
read_doc(5000)

http://news.livedoor.com/article/detail/6590044/
2012-05-24T12:55:00+0900
ソニー、Sony TabletシリーズをAndroid 4.0にOSバージョンアップするソフトウェアアップデートを提供開始
本日24日13時から提供開始！ 

ソニーは18日、同社のAndroid搭載タブレット「Sony Tablet」シリーズにおいて最新プラットフォーム「Android 4.0（開発コード名：IceCream Sandwich；ICS）」へOSバージョンアップするためのシステムソフトウェアアップデートを2012年5月24日（木）13：00から提供開始すると発表しています。

今回のアップデートは、Sony Tablet SシリーズおよびSony Tablet Pシリーズを対象に、OSをAndroid 4.0.3へバージョンアップします。当初は4月下旬に予定されていましたが、ソフトウェア開発に時間を要するとして4月27日に延期することが発表されていました。

OSバージョンアップ以外にも“Sony Tablet”独自の機能が追加されるとのことです。



主なアップデート内容は以下の通り。

＜Android 4.0.3に伴う機能追加＞

1. 最新版のAndroid 4.0.3にアップデートされることにより、Android 4.0対応スマートフォンとユーザーインターフェースが共通化します。
2. ロック画面からのカメラの直接起動やパノラマ写真の撮影、「ギャラリー」アプリケーションで画像の編集が可能になるなど、写真・動画についての機能が向上します。


＜“Sony Tablet”独自の機能追加＞
1. ソニー製BDレコーダー※1と連携してテレビ番組の視聴が可能
ソニー製のBDレコーダー※1で録画した番組や放送中の番組を“Sony Tablet”上で視聴することが可能になります。家庭内のネットワークに接続することで、テレビのない寝室やキッチン、書斎など、場所を選ばず家中でテレビ番組をハイビジョン（720p）で楽しむことができます。
新しくソニーが提供するアプリケーション「RECOPLA（レコプラ）」※3を、“Sony Tablet”上の「Select App」※4、またはGoogle Playストアから無

In [175]:
similar_to(5000, num=10, distance=True)

[(5558, 0.065379147319052699),
 (5297, 0.066569651435818056),
 (5497, 0.067867272056701844),
 (5660, 0.07717944994314696),
 (5349, 0.080324860323738534),
 (5630, 0.083285008567317576),
 (5296, 0.085007544813284874),
 (5539, 0.11145168036414599),
 (4952, 0.11305264390568159),
 (5287, 0.11529363722508312)]

In [176]:
read_doc(5558)

http://news.livedoor.com/article/detail/6843206/
2012-08-10T11:55:00+0900
KDDI、au向け「Optimus X IS11LG」にAndroid 4.0 ICSにOSバージョンアップするなどのためのソフトウェア更新を提供開始
Optimus X IS11LGがAndroid 4.0に！ 

KDDIおよび沖縄セルラーは10日、コンパクトながらデュアルコアCPUなどと搭載したハイスペックなau向けAndroidスマートフォン「Optimus X IS11LG」（LGエレクトロニクス製）においてAndroid 4.0（開発コード名：IceCream Sandwich；ICS）へのOSバージョンアップを含む本体ファームウェアの更新サービス「OSアップデート」を提供開始したことをお知らせしています。

今回のアップデートで変更されるのは主に以下の4点です。Optimus X IS11LGのOSアップデートははじめてで、更新にかかる時間は約30分、アップデートファイルサイズは約331MBです。ただし、アップデートに必要な時間はユーザーの利用環境によって異なる場合があるということです。

なお、アップデートはパソコンを用いて行い、MacOSには非対応で、本体のみで3G回線またはWi-Fi回線を利用したアップデートはできないので注意してください。

・Android 4.0
※ Android 4.0更新内容の一部をご案内いたします。詳しくは、LG社ホームページよりご確認ください。
　　LG社ホームページ
　1. ロック画面が全方向に解除可能となります。また、ロック画面の下段にショートカットが追加され、ロック画面から直接各アプリを起動可能となります。
　2. ショートカット、ウィジェット、フォルダのサイズ変更が可能になります。
　3. アプリ、ダウンロード、ウィジェットの各タブと編集ボタンが追加されます。また、フォルダを作成することが可能になります。
　4. アプリの無効化機能が追加されます。アンインストール不可の一部アプリについて、「無効にする」を選択すると、アプリランチャー上に表示されなくなります。
・Eメール (〜@ezweb.ne.jp) アプリの改善
　1. 背景色の設定を白背景、黒背景のいずれか

In [165]:
read_doc(1410)

http://news.livedoor.com/article/detail/6679843/
2012-06-21T13:00:00+0900
非公式アプリでも結構イケる！ UbuntuでもEvernoteが使える【デジ通】
Ubuntu（ウブントゥ）には多くのアプリが付属するが、それだけでは十分でないこともある。たとえば、人気のクラウドサービスEvernote用のクライアントは提供されていない。

だが、世界中の開発者がアプリを開発し、ほとんどが無償提供されているLinuxの世界。「非公式アプリ」とはいえ互換性のあるアプリが提供されている。今回は、Evernote用のクライアント「NixNote」を紹介しよう。


■「NixNote」をダウンロード
「NixNote」は、下記のサイトからダウンロードする。インストールパッケージは、ホームフォルダの「ダウンロード」に保存されている。

インストールするには、以前のLinuxのように長いコマンドをタイプする必要はない。パッケージのアイコンをダブルクリックするだけだが、途中でパスワードの入力を求められるところだけがWindowsと異なる点だ。インストールは「Ubuntuソフトウェアセンター」が管理している。

■外観は公式アプリにそっくり
最初の起動は、ランチャー最上部にある「Dashホーム」から行う。JAVAを実行環境としているため、キリンが描かれたスプラッシュ画面があらわれるなど起動は速くはないが、起動後の動作は軽快だ。なお、「NixNote」をランチャーに常時登録したい場合は、起動時にランチャーにあらわれるキリンのアイコンを右クリックして登録する。

外観は公式アプリと大きな違いはなく、ノートの追加や削除も簡単だ。ノートの表示方法は微妙に異なるが、「リスト」「サマリー」「カード」の３種類がしっかり用意されている。リストの場合、標準で「引用元URL」や「作成者」など多数の項目が表示されるが、設定の変更で減らすことも可能だ。

Evernoteのアカウントは、［編集］→［設定］で設定する。同期を自動、15分間隔、手動などに設定できる点も、公式アプリとそん色ない。ただ、Firefoxのアドオンである「Evernote Web Clipper」との連動はできない。

■公式アプリではない点は注意
一通り使ってみたが、

In [166]:
from gensim import similarities

In [167]:
doc_index = similarities.docsim.MatrixSimilarity(lda[corpus])

In [187]:
c = corpus[5000]
vec_lda = lda[c]

In [188]:
s = doc_index.get_similarities(vec_lda)

In [189]:
s = sorted(enumerate(s), key=lambda t: t[1], reverse=True)

In [190]:
s[1:11]

[(5558, 0.37940085),
 (5297, 0.37891358),
 (5497, 0.37838998),
 (5660, 0.37461069),
 (5349, 0.37324414),
 (5630, 0.37213087),
 (5296, 0.3714397),
 (5539, 0.36055902),
 (4952, 0.35997331),
 (5287, 0.35913676)]

In [192]:
similar_to(5000, num=10, distance=True)

[(5558, 0.065379147319052699),
 (5297, 0.066569651435818056),
 (5497, 0.067867272056701844),
 (5660, 0.07717944994314696),
 (5349, 0.080324860323738534),
 (5630, 0.083285008567317576),
 (5296, 0.085007544813284874),
 (5539, 0.11145168036414599),
 (4952, 0.11305264390568159),
 (5287, 0.11529363722508312)]